## BRONZE TO SILVER LAYER

### SILVER Layer - Process ProfitLoss History


In [5]:
# Import necessary libraries and utility functions
import pandas as pd
from common_utilities import global_path, logger

### Data Processing

- Read the trade history data from the Gold layer.
- Filter, transform, and calculate new columns.
- Save the processed data to the Silver layer.


In [6]:
try:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(global_path.tradehistory_gold_file_path)
    logger.info(
        f"Read GOLD Layer trade history data from: {global_path.tradehistory_gold_file_path}"
    )
except Exception as e:
    logger.error(
        f"Failed to read GOLD Layer trade history data due to error: {e}"
    )

2024-08-01T13:56:37Z - INFO - Read GOLD Layer trade history data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\TradeHistory\TradeHistory_data.csv


In [7]:
# Convert 'datetime' column to datetime objects
df["datetime"] = pd.to_datetime(df["datetime"])

# Filter out rows where 'pnl_amount' is zero
df = df[df["pnl_amount"] != 0]

# Update 'side' column: 'SELL' becomes 'LONG', 'BUY' becomes 'SHORT'
df["side"] = df["side"].apply(
    lambda x: "LONG" if x == "SELL" else "SHORT" if x == "BUY" else x
)

# Rename columns for clarity
df = df.rename(
    columns={
        "price": "close_price",
        "avg_price": "open_price",
    }
)

# Calculate open and close side amounts
df["open_amount"] = df["open_price"] * df["quantity"]
df["close_amount"] = df["close_price"] * df["quantity"]

# Sort the DataFrame by 'segment', 'stock_name', and 'datetime'
df = df.sort_values(by=["segment", "stock_name", "datetime"])

# Round the values in to two decimal places
df = df.round(2)

# Reset index to ensure it starts from 0
df = df.reset_index(drop=True)

In [8]:
try:
    # Select and reorder the columns for the final DataFrame
    df = df[
        [
            "datetime",
            "exchange",
            "segment",
            "stock_name",
            "side",
            "quantity",
            "open_price",
            "open_amount",
            "close_price",
            "close_amount",
            "pnl_amount",
            "pnl_percentage",
        ]
    ]
    # Save the final DataFrame to a CSV file
    df.to_csv(global_path.profitloss_gold_file_path, index=None)
    logger.info("GOLD Layer CSV file for ProfitLoss successfully created at:")
    logger.info(global_path.profitloss_gold_file_path.resolve())
    # Display the DataFrame information
    df.info()
except Exception as e:
    logger.error(f"Failed to save GOLD Layer CSV file due to error: {e}")

2024-08-01T13:56:37Z - INFO - GOLD Layer CSV file for ProfitLoss successfully created at:
2024-08-01T13:56:37Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\ProfitLoss\ProfitLoss_data.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   datetime        84 non-null     datetime64[ns]
 1   exchange        84 non-null     object        
 2   segment         84 non-null     object        
 3   stock_name      84 non-null     object        
 4   side            84 non-null     object        
 5   quantity        84 non-null     float64       
 6   open_price      84 non-null     float64       
 7   open_amount     84 non-null     float64       
 8   close_price     84 non-null     float64       
 9   close_amount    84 non-null     float64       
 10  pnl_amount      84 non-null     float64       
 11  pnl_percentage  84 non-null     float64       
dtypes: datetime64[ns](1), float64(7), object(4)
memory usage: 8.0+ KB


In [9]:
# print(amount)